In [1]:
import os
from glob import glob
from tqdm import tqdm
from shutil import copy

from src.gradio_demo import SadTalker

from huggingface_hub import snapshot_download

from elevenlabs import generate, save, set_api_key
from moviepy.editor import VideoFileClip

from PIL import Image
from rembg import remove

set_api_key('28df0b632b692bda62e579cb616a5266')

In [2]:
snapshot_download(repo_id='vinthony/SadTalker-V002rc', local_dir='./checkpoints', local_dir_use_symlinks=True)

sad_talker = SadTalker(lazy_load=True)

Fetching 7 files:   0%|          | 0/7 [00:00<?, ?it/s]

In [3]:
kosakata = open("kosakata.txt", "r").read().split("\n")
kosakata

['a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 'ba',
 'bi',
 'bu',
 'be',
 'bo',
 'ca',
 'ci',
 'cu',
 'ce',
 'co',
 'da',
 'di',
 'du',
 'de',
 'do',
 'fa',
 'fi',
 'fu',
 'fe',
 'fo',
 'ga',
 'gi',
 'gu',
 'ge',
 'go',
 'ha',
 'hi',
 'hu',
 'he',
 'ho',
 'ja',
 'ji',
 'ju',
 'je',
 'jo',
 'ka',
 'ki',
 'ku',
 'ke',
 'ko',
 'la',
 'li',
 'lu',
 'le',
 'lo',
 'ma',
 'mi',
 'mu',
 'me',
 'mo',
 'na',
 'ni',
 'nu',
 'ne',
 'no',
 'pa',
 'pi',
 'pu',
 'pe',
 'po',
 'qa',
 'qi',
 'qu',
 'qe',
 'qo',
 'ra',
 'ri',
 'ru',
 're',
 'ro',
 'sa',
 'si',
 'su',
 'se',
 'so',
 'ta',
 'ti',
 'tu',
 'te',
 'to',
 'va',
 'vi',
 'vu',
 've',
 'vo',
 'wa',
 'wi',
 'wu',
 'we',
 'wo',
 'xa',
 'xi',
 'xu',
 'xe',
 'xo',
 'ya',
 'yi',
 'yu',
 'ye',
 'yo',
 'za',
 'zi',
 'zu',
 'ze',
 'zo']

In [15]:
for word in tqdm(kosakata):
    filename = f"audios/{word}.wav"
    
    audio = generate(
            text="<break time='1s' />" + word + "<break time='1s' />",
            model="eleven_multilingual_v2"
        )

    save(
        audio=audio,
        filename=filename
    )

100%|██████████| 21/21 [00:29<00:00,  1.38s/it]


In [ ]:
failed_kosakata = []
failed_audio = []

for audio in glob("audios/*"):
    filename = audio.split("/")[-1].split(".")[0]

    try:
        result = sad_talker.test(
            source_image="images/woman.png",
            driven_audio=audio,
            preprocess='crop',
            still_mode=True, 
            use_enhancer=True, 
            batch_size=1, 
            size=256,
            pose_style=0,
            facerender='facevid2vid',
            exp_scale=1.0,
            use_ref_video=False,
            ref_video=None,
            ref_info=None,
            use_idle_mode=False,
            length_of_audio=0, 
            use_blink=True,
            result_dir='./results/'
        )
    
        copy(result, f"videos/{filename}.mp4")
    except:
        failed_kosakata.append(filename)
        failed_audio.append(audio)

In [ ]:
failed_video = []

for video in glob("videos/*"):
    try:
        filename = video.split("/")[-1].split(".")[0]
        
        videoClip = VideoFileClip(video)
        videoClip.write_gif(f"gifs/{filename}.gif")
    except:
        failed_video.append(video)

In [7]:
size = (512, 512)

for gif in tqdm(glob("gifs/*")):
    filename = gif.split("/")[-1].split(".")[0]
    folder_path = f"sheets/512/{filename}"
    
    os.makedirs(folder_path, exist_ok=True)
    
    im = Image.open(gif)

    for frame in range(0, im.n_frames):
        im.seek(frame)

        extracted_frame = im.resize(size)
        # extracted_frame = remove(extracted_frame)

        extracted_frame.convert('RGB').save(f"{folder_path}/{filename}_{frame}.jpg")

100%|██████████| 110/110 [00:03<00:00, 28.71it/s]


## Fail Safe

In [25]:
for word in tqdm(failed_kosakata_again):
    filename = f"audios/{word}.wav"
    
    audio = generate(
            text="<break time='1s' />" + word + "<break time='1s' />",
            model="eleven_multilingual_v2"
        )

    save(
        audio=audio,
        filename=filename
    )

100%|██████████| 4/4 [00:05<00:00,  1.37s/it]


In [26]:
# failed_kosakata_again = []
# failed_audio_again = []
# 
# succeed_audio = []

for audio in failed_audio_again:
    filename = audio.split("/")[-1].split(".")[0]
    
    try:
        result = sad_talker.test(
                source_image="images/woman.png",
                driven_audio=audio,
                preprocess='crop',
                still_mode=True, 
                use_enhancer=True, 
                batch_size=1, 
                size=256,
                pose_style=0,
                facerender='facevid2vid',
                exp_scale=1.0,
                use_ref_video=False,
                ref_video=None,
                ref_info=None,
                use_idle_mode=False,
                length_of_audio=0, 
                use_blink=True,
                result_dir='./results/'
            )
    
        copy(result, f"videos/{filename}.mp4")
        
        succeed_audio.append(f"videos/{filename}.mp4")
    except:
        # failed_kosakata_again.append(filename)
        # failed_audio_again.append(audio)
        pass

using safetensor as default
{'checkpoint': 'checkpoints/SadTalker_V0.0.2_256.safetensors', 'dir_of_BFM_fitting': 'src/config', 'audio2pose_yaml_path': 'src/config/auido2pose.yaml', 'audio2exp_yaml_path': 'src/config/auido2exp.yaml', 'pirender_yaml_path': 'src/config/facerender_pirender.yaml', 'pirender_checkpoint': 'checkpoints/epoch_00190_iteration_000400000_checkpoint.pt', 'use_safetensor': True, 'mappingnet_checkpoint': 'checkpoints/mapping_00229-model.pth.tar', 'facerender_yaml': 'src/config/facerender.yaml'}
images/woman.png


Face Renderer:: 100%|██████████| 20/20 [00:03<00:00,  5.70it/s]


The generated video is named ./results/efb1321d-bd82-407c-8fe6-7d9d481895fe/woman##mi.mp4
face enhancer....


Face Enhancer:: 100%|██████████| 20/20 [00:03<00:00,  5.34it/s]


The generated video is named ./results/efb1321d-bd82-407c-8fe6-7d9d481895fe/woman##mi_enhanced.mp4
The generated video is named woman##mi in ./results/efb1321d-bd82-407c-8fe6-7d9d481895fe
using safetensor as default
{'checkpoint': 'checkpoints/SadTalker_V0.0.2_256.safetensors', 'dir_of_BFM_fitting': 'src/config', 'audio2pose_yaml_path': 'src/config/auido2pose.yaml', 'audio2exp_yaml_path': 'src/config/auido2exp.yaml', 'pirender_yaml_path': 'src/config/facerender_pirender.yaml', 'pirender_checkpoint': 'checkpoints/epoch_00190_iteration_000400000_checkpoint.pt', 'use_safetensor': True, 'mappingnet_checkpoint': 'checkpoints/mapping_00229-model.pth.tar', 'facerender_yaml': 'src/config/facerender.yaml'}
images/woman.png


...

Face Renderer:: 100%|██████████| 20/20 [00:03<00:00,  5.43it/s]


The generated video is named ./results/7b53a836-02a0-4c13-b868-4b43a523fb90/woman##ya.mp4
face enhancer....


Face Enhancer:: 100%|██████████| 20/20 [00:04<00:00,  4.87it/s]


The generated video is named ./results/7b53a836-02a0-4c13-b868-4b43a523fb90/woman##ya_enhanced.mp4
The generated video is named woman##ya in ./results/7b53a836-02a0-4c13-b868-4b43a523fb90
using safetensor as default
{'checkpoint': 'checkpoints/SadTalker_V0.0.2_256.safetensors', 'dir_of_BFM_fitting': 'src/config', 'audio2pose_yaml_path': 'src/config/auido2pose.yaml', 'audio2exp_yaml_path': 'src/config/auido2exp.yaml', 'pirender_yaml_path': 'src/config/facerender_pirender.yaml', 'pirender_checkpoint': 'checkpoints/epoch_00190_iteration_000400000_checkpoint.pt', 'use_safetensor': True, 'mappingnet_checkpoint': 'checkpoints/mapping_00229-model.pth.tar', 'facerender_yaml': 'src/config/facerender.yaml'}
images/woman.png


mel:: 100%|██████████| 21/21 [00:00<00:00, 77263.49it/s]


In [ ]:
succeed_video = []

for video in tqdm(succeed_audio):
    try:
        filename = video.split("/")[-1].split(".")[0]
        
        videoClip = VideoFileClip(video)
        videoClip.write_gif(f"gifs/{filename}.gif")
        
        succeed_video.append(f"gifs/{filename}.gif")
    except:
        failed_video.append(video)

In [29]:
size = (512, 512)

for gif in tqdm(succeed_video):
    filename = gif.split("/")[-1].split(".")[0]
    folder_path = f"sheets/512/{filename}"
    
    os.makedirs(folder_path, exist_ok=True)
    
    im = Image.open(gif)

    for frame in range(0, im.n_frames):
        im.seek(frame)

        extracted_frame = im.resize(size)
        # extracted_frame = remove(extracted_frame)

        extracted_frame.convert('RGB').save(f"{folder_path}/{filename}_{frame}.jpg")

100%|██████████| 22/22 [00:00<00:00, 27.82it/s]
